In [330]:
import pandas as pd
import os
from dotenv import load_dotenv
import statsmodels.api as sm
import numpy as np

In [331]:

load_dotenv()

True

# Market Income Regression

In [332]:
current_dir = os.getcwd()
data_dir = os.getenv("DATA_PATH")

file_path = os.path.join(data_dir, "Statistics Canada", "incomes_quintile_hh.xlsx")
df = pd.read_excel(file_path)

In [333]:
df["log_comp_all"] = np.log(df["comp_all"])
# Define independent (compx) and dependent (comp) variables
compx = df[["log_comp_all"]]  # Log of total income as independent variable

# Add a constant term (intercept)
compx = sm.add_constant(compx)

In [334]:
for i in range(1, 6):  
    quantile_col = f"comp_q{i}"
    log_quantile_col = f"log_comp_q{i}"
    
    if quantile_col not in df.columns:
        print(f"Column {quantile_col} not found in DataFrame.")
        continue  # Skip if the column is missing

    # Handle zero or negative values before log transformation
    if (df[quantile_col] <= 0).any():
        print(f"Skipping {quantile_col} due to zero/negative values.")
        continue

    # Compute log for the current quantile
    df[log_quantile_col] = np.log(df[quantile_col])

    # Define dependent variable (compy)
    compy = df[log_quantile_col]
    
    # Run the regression
    model = sm.OLS(compy, compx).fit()
    
    # Print results
    print(f"Quintile {i}")
    print(model.summary())
    print("\n" + "="*80 + "\n")


Quintile 1
                            OLS Regression Results                            
Dep. Variable:            log_comp_q1   R-squared:                       0.914
Model:                            OLS   Adj. R-squared:                  0.910
Method:                 Least Squares   F-statistic:                     254.2
Date:                Fri, 10 Oct 2025   Prob (F-statistic):           2.85e-14
Time:                        09:28:56   Log-Likelihood:                 41.701
No. Observations:                  26   AIC:                            -79.40
Df Residuals:                      24   BIC:                            -76.89
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.5008      0.540   

# Apply Growth Rates for 2022 & 2023

In [335]:
#calculate growth rates 

for i in range(1, 6):
    df[f"compq{i}_g"] = df[f"comp_q{i}"].pct_change()
    df[f"tranrq{i}_g"] = df[f"tranr_q{i}"].pct_change()


In [336]:
#store growth rates for 2022 and 2023 in a dictionary

vars = ['compq1', 'compq2', 'compq3', 'compq4', 'compq5', 'tranrq1', 'tranrq2', 'tranrq3', 'tranrq4', 'tranrq5']
growth_rates = {}  # Dictionary to store variable names and values

for var in vars:
    # Dynamically fetch values for 2022 and 2023 and assign them to keys in the dictionary
    growth_rates[f'canada_{var}_2022'] = df.loc[df['year'] == 2022, f'{var}_g'].values[0]
    growth_rates[f'canada_{var}_2023'] = df.loc[df['year'] == 2023, f'{var}_g'].values[0]
    growth_rates[f'canada_{var}_2024'] = df.loc[df['year'] == 2024, f'{var}_g'].values[0]

# Now print the stored results from the dictionary
for key, value in growth_rates.items():
    print(f'{key}: {value}')

canada_compq1_2022: 0.012403379471508158
canada_compq1_2023: 0.05779474431818188
canada_compq1_2024: 0.012589173310952662
canada_compq2_2022: 0.039472560020505254
canada_compq2_2023: 0.028740513438724413
canada_compq2_2024: 0.0682859273463301
canada_compq3_2022: 0.06814698983580914
canada_compq3_2023: 0.023233003454939283
canada_compq3_2024: 0.06461120251806274
canada_compq4_2022: 0.07878008173149698
canada_compq4_2023: 0.023342686776569588
canada_compq4_2024: 0.046177443402855234
canada_compq5_2022: 0.07499350214843847
canada_compq5_2023: 0.04678224188487068
canada_compq5_2024: 0.0237315956045685
canada_tranrq1_2022: -0.0039343480507093576
canada_tranrq1_2023: -0.006742738589211594
canada_tranrq1_2024: 0.03779875477003425
canada_tranrq2_2022: -0.02424912235079968
canada_tranrq2_2023: 0.004563928309680776
canada_tranrq2_2024: 0.03405737025368927
canada_tranrq3_2022: -0.07645217869793997
canada_tranrq3_2023: 0.04148357786549495
canada_tranrq3_2024: 0.036434496567505636
canada_tranrq4_20

In [337]:
#import 2021 census, filter data


file_path_census = os.path.join(data_dir, "Statistics Canada", "Census 2021", "Hierarchical", "censush.csv")
census2021 = pd.read_csv(file_path_census)
census2021 = census2021[census2021["PR"] == 35]

# Drop rows where income equals 88888888 (dropping NA values)
census2021 = census2021[census2021["TOTINC"] != 88888888]
census2021 = census2021[census2021["MRKINC"] != 88888888]
census2021 = census2021[census2021["GTRFS"] != 88888888]
census2021 = census2021[census2021["TOTINC_AT"] != 88888888]

In [338]:
# List of columns to drop
wt_columns = [f"WT{i}" for i in range(1, 17)]

columns_to_drop = [
    "ABOID", "AGEIMM", "BFNMEMB", "BUILT", "CF_RP", "CFSTRUCT", "CIP2021", "CITIZEN", "CITOTH",
    "CONDO", "COW", "DIST", "DTYPE", "EF_RP", "EFCOVID_ERB", "EFDECILE", "EFDIMBM_2018", "EMPIN", 
    "ETHDER", "FCOND", "FOL", "FPTWK", "GENDER", "GENSTAT", "HDGREE", "HLMOSTEN", 
    "HLMOSTFR", "HLMOSTNO", "HLREGEN", "HLREGFR", "HLREGNO", "HRSWRK", "INCTAX", "JOBPERM", "KOL", 
   "LI_ELIG_OML_U18", "LICO_AT", "LICO_BT", "LOC_ST_RES", "LOCSTUD", "LOLIMA", "LOLIMB", 
    "LOMBM_2018", "LSTWRK", "LWMOSTEN", "LWMOSTFR", "LWMOSTNO", "LWREGEN", "LWREGFR", "LWREGNO", 
    "MARSTH", "MOB1", "MOB5", "MODE"
]

# Drop the columns
census2021 = census2021.drop(columns=wt_columns)
census2021 = census2021.drop(columns=columns_to_drop, errors="ignore")  # 'errors="ignore"' ensures no error if a column is missing


#set IMMSTAT = 1 if equal to 8
census2021["IMMSTAT"] = census2021["IMMSTAT"].replace(8, 1)

#create suitable bedroom variavle


# Step 1: Define Household-Level Calculation
def household_bedsuit(group):
    num_couples = (group['CFSTAT'] == 1).sum() // 2 + (group['CFSTAT'] == 2).sum() // 2 # Couples share a room
    num_single_parents = (group['CFSTAT'] == 3).sum()  # Each single parent gets a room
    num_children = (group['CFSTAT'].isin([4, 5])).sum()  # Count children
    num_non_family = (group['CFSTAT'].isin([7, 8])).sum()  # Each gets their own room
    num_living_alone = (group['CFSTAT'] == 6).sum()  # People living alone

    # If the household has exactly 1 person and they live alone → Assign bedsuit = 0 (bachelor unit)
    if len(group) == 1 and num_living_alone == 1:
        return 0  

    # Start with bedrooms for couples and single parents
    bedrooms_needed = num_couples + num_single_parents

    # Assign bedrooms for children: Every 2 children share 1 room
    if num_children > 0:
        bedrooms_needed += (num_children + 1) // 2  # Round up when odd number of children

    # Add rooms for non-family members (CFSTAT = 7, 8)
    bedrooms_needed += num_non_family

    # If NOS == 1, ensure bedsuit does NOT exceed BEDRM
    if 'NOS' in group.columns and group['NOS'].eq(1).any():  # Ensure 'NOS' exists
        max_bedrooms = group['BEDRM'].dropna().max()  # Get the max BEDRM in household
        if pd.notna(max_bedrooms):  
            bedrooms_needed = min(bedrooms_needed, max_bedrooms)  # Ensure it doesn't exceed BEDRM

    return bedrooms_needed

# Step 2: **Initialize `bedsuit` column with NaN**
census2021['bedsuit'] = pd.NA

# Step 3: Apply household-level logic
census2021['bedsuit'] = census2021.groupby('HH_ID')['bedsuit'].transform(
    lambda x: household_bedsuit(census2021.loc[x.index])
)

In [339]:
#create unemployment indicator
def assign_econ(row):
    if row['AGEGRP'] in [1, 2]:
        return 3
    elif 2 < row['LFACT'] <= 10 and row['AGEGRP'] >= 3:
        return 1
    else:
        return 0

# Apply the function to create the ECON variable in census21
census2021['econ'] = census2021.apply(assign_econ, axis=1)


In [340]:


# Build the full path to amr2021.xlsx
file_path = os.path.join(data_dir, "amr2021.xlsx")


#create a median market rent variable for Toronto/not-Toronto, by bedrooms for 2021
amr2021 = pd.read_excel(file_path)
# Perform a left join to add the MMR column to census2021
census2021 = census2021.merge(amr2021, on=['bedsuit', 'CMA'], how='left')

census2021['mmr'] = census2021['mmr'] 

In [341]:
# Step 1: Correct missing values (children with no income)
census2021.loc[census2021["TOTINC"] > 88000000, "TOTINC"] = 0
census2021.loc[census2021["MRKINC"] > 88000000, "MRKINC"] = 0
census2021.loc[census2021["GTRFS"] > 88000000, "GTRFS"] = 0
census2021.loc[census2021["TOTINC_AT"] > 88000000, "TOTINC_AT"] = 0

#check missing and NA values
variables = ["TOTINC", "MRKINC", "GTRFS"]
for var in variables:
    count = (census2021[var] > 88000000).sum()
    print(f"Number of records with {var} > 88000000: {count}")

census2021["totaltransfers"] = census2021["GTRFS"]

Number of records with TOTINC > 88000000: 0
Number of records with MRKINC > 88000000: 0
Number of records with GTRFS > 88000000: 0


create quintiles

In [342]:
# Step 1: Create a new column for total income
census2021["totalincome"] = census2021["MRKINC"] + census2021["GTRFS"]

# Step 2: Compute total household income (sum of MRKINC and GTRFS)
household_income = census2021.groupby("HH_ID").agg(
    totalincome=("totalincome", "sum"),  # Sum of the newly created totalincome
    weight=("WEIGHT", "first")  # Take first weight per household
).reset_index()


# Step 3: Compute weighted quintiles
def weighted_quantile(values, quantiles, sample_weight):
    """Calculate weighted quantiles for a given dataset."""
    sorter = np.argsort(values)
    values, sample_weight = values[sorter], sample_weight[sorter]
    weighted_cdf = np.cumsum(sample_weight) / np.sum(sample_weight)
    return np.interp(quantiles, weighted_cdf, values)

# Compute cutoff points for weighted quintiles
quantile_cutoffs = weighted_quantile(
    household_income["totalincome"].values,
    quantiles=[0.2, 0.4, 0.6, 0.8],
    sample_weight=household_income["weight"].values
)

# Step 4: Assign households to weighted quintiles
household_income["quintile"] = np.digitize(household_income["totalincome"], bins=quantile_cutoffs, right=True) + 1

# Step 5: Merge the quintile info back to the original dataset
census2021 = census2021.merge(household_income[["HH_ID", "quintile"]], on="HH_ID", how="left")

# Step 6: Verify the result
print(census2021[["HH_ID", "TOTINC", "quintile"]].head(10))

#subset census2021 for only quintiles 1 and 2
census2021 = census2021[census2021["quintile"].isin([1, 2, 3])]


   HH_ID  TOTINC  quintile
0      1   12000         1
1      4   34000         4
2      4   97000         4
3      4       0         4
4      4       0         4
5      4       0         4
6      6    3000         3
7      6    8000         3
8      6   69000         3
9      6       0         3


Calculate 2021 Core Housing Need

In [343]:
# Step 1: Aggregate total household income
household_income2 = census2021.groupby('HH_ID')['totalincome'].sum().reset_index()

# Step 2: Merge back into census2021
census2021 = census2021.merge(household_income2, on='HH_ID', suffixes=('', '_hh'))


# identify households where every preson in unrelated or living alone
household_non_family = census2021.groupby('HH_ID')['CFSTAT'].apply(lambda x: set(x).issubset({6, 7})).reset_index()
household_non_family.columns = ['HH_ID', 'non_family_household']
household_non_family['non_family_household'] = household_non_family['non_family_household'].astype(int)

#merge back into census2021
census2021 = census2021.merge(household_non_family, on='HH_ID', how='left')

#Identify households where at least one maintainer (HHMAINP == 1) is aged 15-29 and attending school
maintainers = census2021[(census2021['HHMAINP'] == 1) & 
                         (census2021['AGEGRP'].isin([3, 4, 5])) & 
                         (census2021['ATTSCH'] == 1)]

excluded_households = maintainers['HH_ID'].unique()
census2021['student_household'] = census2021['HH_ID'].isin(excluded_households).astype(int) # 1 if student, 0 otherwise

# Step 3: Define Core Housing Need (CHN)
census2021['chn'] = 0  # Default to 0 (not in CHN)

# Condition: Housing is unaffordable OR unsuitable OR inadequate
housing_issue = (
    (census2021['SHELCO'] * 12 / census2021['totalincome_hh'] > 0.30) |  # Unaffordable
    (census2021['NOS'] == 0) |  # Unsuitable
    (census2021['REPAIR'] == 3)  # Inadequate
)

# Condition: Alternative market rent is also unaffordable
market_unaffordable = (census2021['mmr']) * 12 > 0.30 * census2021['totalincome_hh']

# Set Core Housing Need (CHN) variable, excluding non-family households with an eligible maintainer
census2021.loc[housing_issue & market_unaffordable & 
               ~((census2021['student_household'] == 1) & (census2021['non_family_household'] == 1)), 
               'chn'] = 1


# Create STIR (Shelter Cost to Income Ratio)
census2021["stir"] = (census2021["SHELCO"] * 12) / census2021["totalincome_hh"]

# Create ALTSTIR (Alternative STIR using AMR)
census2021["altstir"] = (census2021["mmr"] * 12) / census2021["totalincome_hh"]

# Update CHN: Exclude individuals with STIR >= 1
census2021.loc[census2021["stir"] >= 1, "chn"] = 0



# Step 4: Define Deep Core Housing Need (DCHN)
census2021['dchn'] = 0  # Default to 0

deep_housing_issue = (
    (census2021['SHELCO'] * 12 / census2021['totalincome_hh'] > 0.50) |  # Deeply unaffordable
    (census2021['NOS'] == 0) |  # Unsuitable
    (census2021['REPAIR'] == 3)  # Inadequate
)

deep_market_unaffordable = (census2021['mmr']) * 12 > 0.50 * census2021['totalincome_hh']

# Apply DCHN flag with same exclusions
census2021.loc[deep_housing_issue & deep_market_unaffordable &
               ~((census2021['student_household'] == 1) & (census2021['non_family_household'] == 1)), 
               'dchn'] = 1

# Update DCHN: Exclude individuals with STIR >= 1
census2021.loc[census2021["stir"] >= 1, "dchn"] = 0

In [344]:
census2022 = census2021.copy()

In [345]:
# Load the population projection file
file_path_pop = os.path.join(data_dir, "popproj.xlsx")
popproj = pd.read_excel(file_path_pop)

# Filter to 2022 growth rates only
growth_rates_2022 = popproj[['demo', 'agegrp', 'econ', 2022]]

# Create a lookup dictionary: {(agegrp, demo, econ): growth_rate}
growth_rate_lookup = {
    (row['agegrp'], row['demo'], row['econ']): row[2022]
    for _, row in growth_rates_2022.iterrows()
}

# Define the AGEGRP mapping from numbers to labels
agegrp_mapping = {
    1: '0to9',
    2: '10to14',
    3: '15to19',
    4: '20to24',
    5: '25to29',
    6: '30to34',
    7: '35to39',
    8: '40to44',
    9: '45to49',
    10: '50to54',
    11: '55to64',
    12: '65to74',
    13: '75plus',
    88: "total"
}

# Function to apply growth rates based on AGEGRP, IMMSTAT, and ECON
def apply_growth(row):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']

    # Map AGEGRP code to string label
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Skip updating this row

    # Determine if the person is NPR or non-NPR based on IMMSTAT
    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Lookup the growth rate, defaulting to 1 if not found
    growth_rate = growth_rate_lookup.get((agegrp_label, demo, econ), 1)
    
    # Apply the growth rate
    row['WEIGHT'] *= (1 + growth_rate)
    return row

# Apply the updated growth function to census2022
census2022 = census2022.apply(apply_growth, axis=1)

summary statistics:
income quintile ranges
number of households per quintile
number of household in CHN per quintile

In [346]:
# Show the min and max household income for each quintile
print(household_income.groupby("quintile")["totalincome"].agg(["min", "max"]))

# Step 1: Aggregate weights at the household level
household_weights = census2021.groupby("HH_ID")["WEIGHT"].first().reset_index()

# Step 2: Merge back the quintile information
household_weights = household_weights.merge(
    household_income[["HH_ID", "quintile"]], on="HH_ID", how="left"
)

# Step 3: Compute weighted household counts by quintile
household_weighted_counts = household_weights.groupby("quintile")["WEIGHT"].sum()

# Print results with comma separators
print(household_weighted_counts.apply(lambda x: f"{x:,.0f}"))

# Sum total weights across all households
total_weight = household_weighted_counts.sum()
print(f"Total households (weighted): {total_weight:,.0f}")

             min      max
quintile                 
1         -38300    44300
2          44301    74500
3          74501   109900
4         110000   162101
5         162200  1613152
quintile
1    1,104,493
2    1,104,192
3    1,102,284
Name: WEIGHT, dtype: object
Total households (weighted): 3,310,969


In [347]:
# Keep one row per household in core housing need
core_housing_need_households = (
    census2021[census2021["HCORENEED_IND"] == 100]
    .drop_duplicates(subset="HH_ID")
)

# Sum of household weights by quintile
core_housing_need_weighted = (
    core_housing_need_households.groupby("quintile")["WEIGHT"]
    .sum()
)

# Format numbers with comma separators
print(core_housing_need_weighted.apply(lambda x: f"{x:,.0f}"))

#print sum of households in core housing need
total_core_housing_need = core_housing_need_weighted.sum()
print(total_core_housing_need)

quintile
1    467,645
2    166,994
3      5,824
Name: WEIGHT, dtype: object
640463.50217058


grow market income for 2022 based on Canada quintile data

In [348]:
def adjust_mrkinc(row):
    if row['quintile'] == 1:
        return row['MRKINC'] * (1 + growth_rates['canada_compq1_2022'])*0.9982
    elif row['quintile'] == 2:
        return row['MRKINC'] * (1 + growth_rates['canada_compq2_2022'])*0.9937
    elif row['quintile'] == 3:
        return row['MRKINC'] * (1 + growth_rates['canada_compq3_2022'])*0.9960
    elif row['quintile'] == 4:
        return row['MRKINC'] * (1 + growth_rates['canada_compq4_2022'])
    elif row['quintile'] == 5:
        return row['MRKINC'] * (1 + growth_rates['canada_compq5_2022'])
    else:
        return row['MRKINC']  # Default case if quintile is missing

# Apply function row-wise
census2022.loc[:, 'MRKINC'] = census2022.apply(adjust_mrkinc, axis=1).astype('float64')

grow transfer income for 2022 based on Canada quintile data

In [349]:
def adjust_gtrfs(row):
    if row['quintile'] == 1:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq1_2022'])*1.0045
    elif row['quintile'] == 2:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq2_2022'])*1.0097
    elif row['quintile'] == 3:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq3_2022'])*1.0116
    else:
        return row['GTRFS']  # Default case if quintile is missing

# Apply function row-wise
census2022.loc[:, 'GTRFS'] = census2022.apply(adjust_gtrfs, axis=1).astype('float64')

In [350]:
#new variable for total income (after growing MRKINC and GTRFS)
census2022["totalincome"] = census2022["MRKINC"] + census2022["GTRFS"]
census2022["totaltransfers"] = census2022["GTRFS"]

In [351]:
# Weighted mean function
def weighted_mean(x, w):
    return np.sum(x * w) / np.sum(w)

# Group by quintile and compute weighted average of totalincome
avg_income_by_quintile = (
    census2022
    .groupby("quintile")
    .apply(lambda g: weighted_mean(g["totalincome"], g["WEIGHT"]))
)

# Convert to DataFrame for nicer display
avg_income_by_quintile = avg_income_by_quintile.reset_index(name="avg_totalincome")

print(avg_income_by_quintile)


   quintile  avg_totalincome
0       1.0     18488.901394
1       2.0     28881.551514
2       3.0     36907.454625


C:\Users\mgordon\AppData\Local\Temp\ipykernel_31512\2768629471.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: weighted_mean(g["totalincome"], g["WEIGHT"]))


grow 2022 shelco based on tenur

In [352]:
print(census2022[['SHELCO', 'mmr']].head()) #check values before growing

   SHELCO     mmr
0   200.0   900.0
1   800.0  1700.0
2   800.0  1700.0
3   800.0  1700.0
4   800.0  1700.0


In [353]:


file_path_growth = os.path.join(data_dir, "growthrates.xlsx")
growth = pd.read_excel(file_path_growth)

print(growth.head())

#increase shelco variable by the 2022 rent variable in the growth df, if tenur is 2, increase by the mortgage variable if tenur is 1

year_to_use = 2022 

def adjust_shelco(row):
    rent = growth.loc[growth['year'] == year_to_use, 'rent'].values[0]
    mortgage = growth.loc[growth['year'] == year_to_use, 'mortgage'].values[0]
    othercosts = growth.loc[growth['year'] == year_to_use, 'othercosts'].values[0]

    if row['TENUR'] == 2:  # Renters
        return row['SHELCO'] * (1 + rent)
    elif row['TENUR'] == 1:
        if row['PRESMORTG'] in [1, 8]:  # Mortgage holders
            return row['SHELCO'] * (1 + mortgage)
        elif row['PRESMORTG'] == 0:  # No mortgage, use othercosts
            return row['SHELCO'] * (1 + othercosts)
    
    # Default case
    return row['SHELCO'] * (1 + rent)
#assigns unknown TENUR to renters

# Apply
census2022['SHELCO'] = census2022.apply(adjust_shelco, axis=1)
print(census2022[['SHELCO', 'mmr']].head())  # Check values after growing

   year      rent  mortgage  othercosts
0  2022  0.052632  0.078221    0.028594
1  2023  0.071429  0.102674    0.033550
2  2024  0.053333  0.041514    0.042860
3  2025  0.035739  0.035362    0.035998
4  2026  0.026306  0.045493    0.034412
       SHELCO     mmr
0  210.526316   900.0
1  822.875193  1700.0
2  822.875193  1700.0
3  822.875193  1700.0
4  822.875193  1700.0


In [354]:
#increase mmr variable by the 2022 rent variable in the growth df
census2022['mmr'] = census2022['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])
print(census2022[['SHELCO', 'mmr']].head()) #check values after growing

       SHELCO          mmr
0  210.526316   947.368421
1  822.875193  1789.473684
2  822.875193  1789.473684
3  822.875193  1789.473684
4  822.875193  1789.473684


create 2023 and 2024 census df, apply population adjustments, market and transfer income growth rates, shelter costs growth rates

In [355]:
# Creating 2023 microsimulation
census2023 = census2022.copy()
# Verify the result
print(census2023[["HH_ID", "PP_ID", "TOTINC", "quintile"]].head(10))

#adj weights for pop growth
# Filter to 2023 growth rates, now including 'econ'
growth_rates_2023 = popproj[['demo', 'agegrp', 'econ', 2023]]
# Create the 2023 growth rate lookup dictionary
growth_rate_lookup_2023 = {
    (row['agegrp'], row['demo'], row['econ']): row[2023]
    for _, row in growth_rates_2023.iterrows()
}
# Apply the 2023 growth rates
def apply_growth_2023(row):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']  # Include ECON in the logic
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Skip updating this row
    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Lookup the growth rate, defaulting to 1 if not found
    growth_rate = growth_rate_lookup_2023.get((agegrp_label, demo, econ), 1)
    
    row['WEIGHT'] *= (1 + growth_rate)
    return row
# Apply the updated function to census2023
census2023 = census2023.apply(apply_growth_2023, axis=1)

def adjust_mrkinc(row):
    if row['quintile'] == 1:
        return row['MRKINC'] * (1 + growth_rates['canada_compq1_2023'])*0.9973
    elif row['quintile'] == 2:
        return row['MRKINC'] * (1 + growth_rates['canada_compq2_2023'])*0.9985
    elif row['quintile'] == 3:
        return row['MRKINC'] * (1 + growth_rates['canada_compq3_2023'])*0.9996
    elif row['quintile'] == 4:
        return row['MRKINC'] * (1 + growth_rates['canada_compq4_2023'])
    elif row['quintile'] == 5:
        return row['MRKINC'] * (1 + growth_rates['canada_compq5_2023'])
    else:
        return row['MRKINC']  # Default case if quintile is missing
# Apply function row-wise
census2023.loc[:, 'MRKINC'] = census2023.apply(adjust_mrkinc, axis=1).astype('float64')

def adjust_gtrfs(row):
    if row['quintile'] == 1:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq1_2023'])*1.0125
    elif row['quintile'] == 2:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq2_2023'])*1.0028
    elif row['quintile'] == 3:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq3_2023'])*1.0020
    elif row['quintile'] == 4:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq4_2023'])
    elif row['quintile'] == 5:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq5_2023'])
    else:
        return row['GTRFS']  # Default case if quintile is missing
# Apply function row-wise
census2023.loc[:, 'GTRFS'] = census2023.apply(adjust_gtrfs, axis=1).astype('float64')

#new variable for total income (after growing MRKINC and GTRFS)
census2023["totalincome"] = census2023["MRKINC"] + census2023["GTRFS"]
census2023["totaltransfers"] = census2023["GTRFS"]

#update shelco & mmr for 2023
year_to_use = 2023  
census2023['SHELCO'] = census2023.apply(adjust_shelco, axis=1)
census2023['mmr'] = census2023['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])
print(census2023[['SHELCO', 'mmr']].head())  # Check values after growing

   HH_ID     PP_ID   TOTINC  quintile
0    1.0   11101.0  12000.0       1.0
1    6.0   61101.0   3000.0       3.0
2    6.0   61102.0   8000.0       3.0
3    6.0   61103.0  69000.0       3.0
4    6.0   61104.0      0.0       3.0
5   13.0  131101.0  63000.0       3.0
6   13.0  131102.0  41000.0       3.0
7   13.0  131103.0      0.0       3.0
8   13.0  131104.0      0.0       3.0
9   15.0  151101.0   5000.0       1.0
       SHELCO          mmr
0  225.563910  1015.037594
1  850.482897  1917.293233
2  850.482897  1917.293233
3  850.482897  1917.293233
4  850.482897  1917.293233


In [356]:
# Creating 2024 microsimulation
census2024 = census2023.copy()
# Verify the result
print(census2024[["HH_ID", "PP_ID", "TOTINC", "quintile"]].head(10))

#adj weights for pop growth
# Filter to 2024 growth rates, now including 'econ'
growth_rates_2024 = popproj[['demo', 'agegrp', 'econ', 2024]]
# Create the 2024 growth rate lookup dictionary
growth_rate_lookup_2024 = {
    (row['agegrp'], row['demo'], row['econ']): row[2024]
    for _, row in growth_rates_2024.iterrows()
}
# Apply the 2024 growth rates
def apply_growth_2024(row):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']  # Include ECON in the logic
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Skip updating this row
    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Lookup the growth rate, defaulting to 1 if not found
    growth_rate = growth_rate_lookup_2024.get((agegrp_label, demo, econ), 1)
    
    row['WEIGHT'] *= (1 + growth_rate)
    return row
# Apply the updated function to census2024
census2024 = census2024.apply(apply_growth_2024, axis=1)

def adjust_mrkinc(row):
    if row['quintile'] == 1:
        return row['MRKINC'] * (1 + growth_rates['canada_compq1_2024'])*0.9946
    elif row['quintile'] == 2:
        return row['MRKINC'] * (1 + growth_rates['canada_compq2_2024'])*0.9967
    elif row['quintile'] == 3:
        return row['MRKINC'] * (1 + growth_rates['canada_compq3_2024'])*0.9989
    elif row['quintile'] == 4:
        return row['MRKINC'] * (1 + growth_rates['canada_compq4_2024'])
    elif row['quintile'] == 5:
        return row['MRKINC'] * (1 + growth_rates['canada_compq5_2024'])
    else:
        return row['MRKINC']  # Default case if quintile is missing
# Apply function row-wise
census2024.loc[:, 'MRKINC'] = census2024.apply(adjust_mrkinc, axis=1).astype('float64')

def adjust_gtrfs(row):
    if row['quintile'] == 1:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq1_2024'])*1.0191
    elif row['quintile'] == 2:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq2_2024'])*1.0056
    elif row['quintile'] == 3:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq3_2024'])*1.0045
    elif row['quintile'] == 4:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq4_2024'])
    elif row['quintile'] == 5:
        return row['GTRFS'] * (1 + growth_rates['canada_tranrq5_2024'])
    else:
        return row['GTRFS']  # Default case if quintile is missing
# Apply function row-wise
census2024.loc[:, 'GTRFS'] = census2024.apply(adjust_gtrfs, axis=1).astype('float64')

#new variable for total income (after growing MRKINC and GTRFS)
census2024["totalincome"] = census2024["MRKINC"] + census2024["GTRFS"]
census2024["totaltransfers"] = census2024["GTRFS"]

#update shelco & mmr for 2024
year_to_use = 2024  
census2024['SHELCO'] = census2024.apply(adjust_shelco, axis=1)
census2024['mmr'] = census2024['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])
print(census2024[['SHELCO', 'mmr']].head())  # Check values after growing

   HH_ID     PP_ID   TOTINC  quintile
0    1.0   11101.0  12000.0       1.0
1    6.0   61101.0   3000.0       3.0
2    6.0   61102.0   8000.0       3.0
3    6.0   61103.0  69000.0       3.0
4    6.0   61104.0      0.0       3.0
5   13.0  131101.0  63000.0       3.0
6   13.0  131102.0  41000.0       3.0
7   13.0  131103.0      0.0       3.0
8   13.0  131104.0      0.0       3.0
9   15.0  151101.0   5000.0       1.0
       SHELCO          mmr
0  237.593985  1069.172932
1  886.934831  2019.548872
2  886.934831  2019.548872
3  886.934831  2019.548872
4  886.934831  2019.548872


In [357]:
# === Step 4: Compute average income growth by quintile ===

def weighted_avg(df, value_col, weight_col):
    """Helper function for weighted average."""
    return (df[value_col] * df[weight_col]).sum() / df[weight_col].sum()

# Prepare results storage
growth_summary = []

for q in sorted(census2022['quintile'].unique()):
    # --- Average weighted incomes by year ---
    avg_mrkinc_2022 = weighted_avg(census2022[census2022['quintile'] == q], 'MRKINC', 'WEIGHT')
    avg_mrkinc_2023 = weighted_avg(census2023[census2023['quintile'] == q], 'MRKINC', 'WEIGHT')
    avg_mrkinc_2024 = weighted_avg(census2024[census2024['quintile'] == q], 'MRKINC', 'WEIGHT')
    
    avg_tran_2022 = weighted_avg(census2022[census2022['quintile'] == q], 'GTRFS', 'WEIGHT')
    avg_tran_2023 = weighted_avg(census2023[census2023['quintile'] == q], 'GTRFS', 'WEIGHT')
    avg_tran_2024 = weighted_avg(census2024[census2024['quintile'] == q], 'GTRFS', 'WEIGHT')
    
    # --- Growth rates ---
    mrkinc_growth_22_23 = (avg_mrkinc_2023 / avg_mrkinc_2022 - 1) * 100
    mrkinc_growth_23_24 = (avg_mrkinc_2024 / avg_mrkinc_2023 - 1) * 100
    
    tran_growth_22_23 = (avg_tran_2023 / avg_tran_2022 - 1) * 100
    tran_growth_23_24 = (avg_tran_2024 / avg_tran_2023 - 1) * 100
    
    growth_summary.append({
        'quintile': q,
        'mrkinc_growth_22_23 (%)': mrkinc_growth_22_23,
        'mrkinc_growth_23_24 (%)': mrkinc_growth_23_24,
        'tran_growth_22_23 (%)': tran_growth_22_23,
        'tran_growth_23_24 (%)': tran_growth_23_24
    })

# Convert to DataFrame for clarity
income_growth_summary = pd.DataFrame(growth_summary)

# Display the summary table
print("=== Average Income Growth by Quintile ===")
display(income_growth_summary)


=== Average Income Growth by Quintile ===


,quintile,mrkinc_growth_22_23 (%),mrkinc_growth_23_24 (%),tran_growth_22_23 (%),tran_growth_23_24 (%)
0,1.0,5.288170,0.210772,-1.100221,4.176068
1,2.0,2.923764,6.117846,0.599708,4.344359
2,3.0,2.507120,5.981994,4.461789,4.856759


In [358]:
# Define actual benefit shares by quintile
benefit_shares = {
    1: {"cpp": 0.25, "ei": 0.025, "oas": 0.329, "child": 0.068, "social": 0.328},
    2: {"cpp": 0.326, "ei": 0.048, "oas": 0.278, "child": 0.127, "social": 0.221},
    3: {"cpp": 0.323, "ei": 0.085, "oas": 0.247, "child": 0.155, "social": 0.190}
}

# Apply benefit shares to census2024
for quintile, shares in benefit_shares.items():
    for benefit, share in shares.items():
        census2024.loc[census2024["quintile"] == quintile, benefit] = (
            share * census2024["GTRFS"]
        )

# (Optional) verify totals
print(census2024.groupby("quintile")[["cpp", "ei", "oas", "child", "social"]].mean().round(2))


              cpp      ei      oas    child   social
quintile                                            
1.0       3220.16  322.02  4237.73   875.88  4224.84
2.0       3643.94  536.53  3107.41  1419.57  2470.28
3.0       2850.61  750.16  2179.88  1367.94  1676.83


In [359]:


# columns we care about
cols = ["quintile", "GTRFS", "cpp", "ei", "oas", "child", "social"]

# pick a single valid row (random for reproducibility)
row = (
    census2024.loc[census2024["GTRFS"].notna(), cols]
    .sample(1, random_state=42)
    .iloc[0]
)

# build a small display table
out = pd.DataFrame({
    "amount": [
        row["GTRFS"], row["cpp"], row["ei"], row["oas"], row["child"], row["social"]
    ],
    "share_of_GTRFS": [
        1.0,
        row["cpp"]   / row["GTRFS"],
        row["ei"]    / row["GTRFS"],
        row["oas"]   / row["GTRFS"],
        row["child"] / row["GTRFS"],
        row["social"]/ row["GTRFS"],
    ]
}, index=["GTRFS", "cpp", "ei", "oas", "child", "social"])

# nice formatting
out_fmt = out.copy()
out_fmt["amount"] = out_fmt["amount"].map(lambda x: f"${x:,.0f}")
out_fmt["share_of_GTRFS"] = (out_fmt["share_of_GTRFS"]*100).map(lambda x: f"{x:.1f}%")

print(f"Quintile for this record: Q{int(row['quintile'])}")
print(out_fmt)

# (Optional) quick integrity check: benefits sum ≈ GTRFS
benefit_sum = row["cpp"] + row["ei"] + row["oas"] + row["child"] + row["social"]
print("\nCheck: benefits sum vs GTRFS")
print(f"Benefits sum: ${benefit_sum:,.0f}")
print(f"GTRFS:        ${row['GTRFS']:,.0f}")
print(f"Diff:         ${benefit_sum - row['GTRFS']:,.0f}")


Quintile for this record: Q1
        amount share_of_GTRFS
GTRFS   $6,492         100.0%
cpp     $1,623          25.0%
ei        $162           2.5%
oas     $2,136          32.9%
child     $441           6.8%
social  $2,129          32.8%

Check: benefits sum vs GTRFS
Benefits sum: $6,492
GTRFS:        $6,492
Diff:         $0


# Export Microsimulations for 2021-2024

In [360]:
#Export Census data (2021 to 2024) to .csv
folder_path = os.path.join(data_dir, "Microsimulations")
os.makedirs(folder_path, exist_ok=True)

census_years = [2021, 2022, 2023, 2024]

# Loop through each year and save the corresponding census dataset
for year in census_years:
    # Dynamically create the census variable name based on the year
    census = globals().get(f"census{year}", None)
    
    if census is not None:
        # Create the full path for saving the CSV file
        csv_filename = f"{folder_path}/census{year}.csv"  # Combine folder path and filename

        # Save the transformed dataset for the current year to a CSV file
        census.to_csv(csv_filename, index=False)  # Save to CSV (without the index)
        print(f"Saved census data for {year} to {csv_filename}")
    else:
        print(f"Dataset for {year} not found.")

Saved census data for 2021 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2021.csv
Saved census data for 2022 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2022.csv
Saved census data for 2023 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2023.csv
Saved census data for 2024 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2024.csv


# 2025 Onwards Growth Rates and Microsimulations

In [361]:
forecast_path = os.path.join(data_dir, "income_growthrates.xlsx")
forecast = pd.read_excel(forecast_path)

#print all forecast df values
print(forecast)


# Creating global object that can be referenced later
for index, row in forecast.iterrows():
    globals()[row['share']] = row['value'] # Assign each row's value to a global variable with the corresponding name

           share     value
0   social_2025g  0.014186
1   social_2026g  0.008713
2   social_2027g  0.009872
3   social_2028g  0.010068
4   social_2029g  0.010196
5   social_2030g  0.010253
6    child_2025g  0.037695
7    child_2026g  0.014918
8    child_2027g  0.015884
9    child_2028g  0.010010
10   child_2029g  0.010060
11   child_2030g  0.011882
12     cpp_2025g  0.026129
13     cpp_2026g  0.020943
14     cpp_2027g  0.020136
15     cpp_2028g  0.019190
16     cpp_2029g  0.019818
17     cpp_2030g  0.020028
18     oas_2025g  0.021007
19     oas_2026g  0.019966
20     oas_2027g  0.019038
21     oas_2028g  0.019971
22     oas_2029g  0.020039
23     oas_2030g  0.019983
24      ei_2025g  0.044031
25      ei_2026g  0.038880
26      ei_2027g  0.035632
27      ei_2028g  0.024933
28      ei_2029g  0.023125
29      ei_2030g  0.022204
30    comp_2025g  0.031334
31    comp_2026g  0.027843
32    comp_2027g  0.034874
33    comp_2028g  0.030395
34    comp_2029g  0.029619
35    comp_2030g  0.028998


In [362]:
popproj = pd.read_excel(file_path_pop)

In [363]:
# Create a lookup dictionary for each year's growth rates, considering ECON
growth_rate_lookups = {}

for year in range(2022, 2031):
    growth_rates_year = popproj[['demo', 'agegrp', 'econ', year]]
    growth_rate_lookups[year] = {
        (row['agegrp'], row['demo'], row['econ']): row[year]
        for _, row in growth_rates_year.iterrows()
    }

def apply_population_growth(row, year):
    agegrp_code = row['AGEGRP']
    immstat = row['IMMSTAT']
    econ = row['econ']  # Ensure ECON is included in calculations
    
    # Map AGEGRP code to string label
    agegrp_label = agegrp_mapping.get(agegrp_code, None)
    
    if agegrp_label is None:
        print(f"Warning: Unknown AGEGRP code {agegrp_code} in row {row.name}")
        return row  # Leave the row unchanged if AGEGRP is invalid

    demo = 'npr' if immstat == 3 else 'nonnpr'
    
    # Get the growth rate for the year, defaulting to 0% (no change) if missing
    growth_rate = growth_rate_lookups[year].get((agegrp_label, demo, econ), 0)
    
    # Apply the growth rate
    row['WEIGHT'] *= (1 + growth_rate)
    return row


In [364]:
# Define the years to loop over
years = range(2025, 2031)  # Covers 2025 to 2030

# Create a dictionary to store census data for each year
census= {"2024": census2024} #start with 2024 census data
weighted_sum_census = {}  # Dictionary to store weighted sum of MRKINC for each year

# Loop through each year
for year in years:
    # Initialize the new year's census data by copying the previous year's data
    prev_year = str(year - 1)
    curr_year = str(year)
    census[curr_year] = census[prev_year].copy()

    census[curr_year] = census[curr_year].apply(lambda row: apply_population_growth(row, year), axis=1)

    year_to_use = year
    census[curr_year]["SHELCO"] = census[curr_year].apply(adjust_shelco, axis=1)
    census[curr_year]['mmr'] = census[curr_year]['mmr']*(1+growth.loc[growth['year'] == year_to_use, 'rent'].values[0])

    # Grow transfer income
    census[curr_year]["ei"] *= (1 + globals()[f"ei_{curr_year}g"])
    census[curr_year]["child"] *= (1 + globals()[f"child_{curr_year}g"])
    census[curr_year]["social"] *= (1 + globals()[f"social_{curr_year}g"])
    census[curr_year]["cpp"] *= (1 + globals()[f"cpp_{curr_year}g"])
    census[curr_year]["oas"] *= (1 + globals()[f"oas_{curr_year}g"])

    # Compute total transfers for the year
    census[curr_year][f"totaltransfers"] = (
        census[curr_year]["ei"] +
        census[curr_year]["child"] +
        census[curr_year]["social"] +
        census[curr_year]["cpp"] +
        census[curr_year]["oas"]
    )

    # Grow market income by Canada-wide shares of total income growth for each quintile
    #canada_compq1 = 0.7834 * globals()[f"comp_{curr_year}g"]
    #canada_compq2 = 0.9956 * globals()[f"comp_{curr_year}g"]
    #canada_compq3 = 1.0034* globals()[f"comp_{curr_year}g"]

    canada_compq1 = 1 * globals()[f"comp_{curr_year}g"]
    canada_compq2 = 1 * globals()[f"comp_{curr_year}g"]
    canada_compq3 = 1* globals()[f"comp_{curr_year}g"]



    census[curr_year].loc[census[curr_year]["quintile"] == 1, "MRKINC"] *= (1 + canada_compq1)
    census[curr_year].loc[census[curr_year]["quintile"] == 2, "MRKINC"] *= (1 + canada_compq2)
    census[curr_year].loc[census[curr_year]["quintile"] == 3, "MRKINC"] *= (1 + canada_compq3)
    
    #create total income variable = transfer income + market income
    census[curr_year]["totalincome"] = census[curr_year]["MRKINC"] + census[curr_year]["totaltransfers"]

    # Create the full path for saving the CSV file
    csv_filename = f"{folder_path}/census{curr_year}.csv"  # Combine folder path and filename

    # Save the transformed dataset for the current year to a CSV file
    census[curr_year].to_csv(csv_filename, index=False)  # Save to CSV (without the index)
    print(f"Saved census data for {curr_year} to {csv_filename}")

Saved census data for 2025 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2025.csv
Saved census data for 2026 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2026.csv
Saved census data for 2027 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2027.csv
Saved census data for 2028 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2028.csv
Saved census data for 2029 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations/census2029.csv
Saved census data for 2030 to C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing

In [365]:
#check weights were applied properly for first record


#print the variables IMMSTAT, AGEGRP, WEIGHT for the first each record in each censusdf then calculate the percent change in the printed WEIGHT value
for year, df in census.items():
    print(f"Year: {year}")
    print(df[['IMMSTAT', 'AGEGRP', 'WEIGHT']].head(1))

#create an array out of each printed WEIGHT value from the previous loop
weights = [df['WEIGHT'].values[0] for df in census.values()]


#print weights array
print(weights)

#calculate the percent change in the values in the weights array
percent_change = [(weights[i] - weights[i - 1]) / weights[i - 1] * 100 for i in range(1, len(weights))]
print(percent_change)

Year: 2024
   IMMSTAT  AGEGRP      WEIGHT
0      1.0    11.0  100.633283
Year: 2025
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  99.819366
Year: 2026
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  99.072799
Year: 2027
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  98.048539
Year: 2028
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  96.737103
Year: 2029
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  95.309341
Year: 2030
   IMMSTAT  AGEGRP     WEIGHT
0      1.0    11.0  94.242491
[np.float64(100.63328345763732), np.float64(99.81936623368422), np.float64(99.07279911313402), np.float64(98.04853896345261), np.float64(96.7371030079933), np.float64(95.30934060998946), np.float64(94.24249055375437)]
[np.float64(-0.8087952573819428), np.float64(-0.74791811320704), np.float64(-1.033845978765351), np.float64(-1.3375374781955207), np.float64(-1.4759201522562246), np.float64(-1.1193551958361567)]


create trace file

In [366]:


# Define the years you want to include
years = range(2021, 2031)

# List to hold second rows
second_rows = []

for year in years:
    file_path = os.path.join(folder_path, f"census{year}.csv")
    
    try:
        # Read the full CSV and select the second row (index 1)
        second_row = pd.read_csv(file_path).iloc[[0]].copy()
        
        # Add a 'year' column
        second_row['year'] = year
        
        # Append to the list
        second_rows.append(second_row)
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue
    except IndexError:
        print(f"File {file_path} does not have a second row.")
        continue

# Combine all second rows into one DataFrame
summary_df = pd.concat(second_rows, ignore_index=True)

# Define the columns you want to keep
columns_to_keep = [
    'year', 'HH_ID', 'PP_ID', 'SHELCO', 'TENUR', 'PRESMORTG', 'AGEGRP', 'quintile', 
    'totalincome', 'totaltransfers', 'social', 'cpp', 
    'ei', 'oas', 'child', 'MRKINC', 'GTRFS', 'WEIGHT', 'IMMSTAT', 'econ'
]

# Keep only the desired columns (ignore missing ones)
summary_df = summary_df[[col for col in columns_to_keep if col in summary_df.columns]]

# Set 'year' as the index
summary_df.set_index('year', inplace=True)

# Transpose the DataFrame
summary_transposed = summary_df.transpose()

# Save the transposed DataFrame to Excel
output_path = os.path.join(folder_path, f"census_trace.xlsx")
summary_transposed.to_excel(output_path)

print(f"Created transposed summary Excel file with years as columns at {output_path}")


Created transposed summary Excel file with years as columns at C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\census_trace.xlsx


In [367]:


# Define input and output folders
input_folder = folder_path
output_folder = os.path.join(folder_path, "household")

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

# Define years for processing
years = range(2021, 2031)  # Covers 2023 to 2030

# Loop through each year's census simulation file
for year in years:
    input_file = f"{input_folder}/census{year}.csv"
    output_file = f"{output_folder}/census{year}_household.csv"
    
    if not os.path.exists(input_file):
        print(f"Skipping {year}: File not found -> {input_file}")
        continue

    # Load individual-level census data
    df = pd.read_csv(input_file)

    # Compute average WEIGHT for adults (AGEGRP > 2)
    adult_weights = df[df["AGEGRP"] > 2].groupby("HH_ID")["WEIGHT"].mean().reset_index()
    adult_weights.rename(columns={"WEIGHT": "avg_adult_weight"}, inplace=True)

    # Group by HH_ID and aggregate
    household_df = df.groupby("HH_ID").agg({
        "BEDRM": "first",
        "HCORENEED_IND": "first",
        "REPAIR": "first",
        "NOS": "first",
        "PRESMORTG": "first",
        "SHELCO": "first",
        "TENUR": "first",
        "SUBSIDY": "first",
        "quintile": "first",
        "totalincome": "sum",
        "MRKINC": "sum",
        "TOTINC_AT": "sum",
        "totaltransfers": "sum",
        "mmr":"first",
        "student_household": "first",
        "non_family_household": "first",
        "chn":"first",
         "dchn":"first",
        "bedsuit":"first",
        "stir" : "first"
    }).reset_index()

   # Merge average adult weight into the household-level dataframe
    household_df = household_df.merge(adult_weights, on="HH_ID", how="left")

   # Rename the merged column to WEIGHT
    household_df.rename(columns={"avg_adult_weight": "WEIGHT"}, inplace=True)

    # Save the transformed household-level data
    household_df.to_csv(output_file, index=False)

    ###To do: gross up weights to align with FAO household projection###
    
    print(f"Saved household-level simulation for {year} -> {output_file}")





Saved household-level simulation for 2021 -> C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\household/census2021_household.csv
Saved household-level simulation for 2022 -> C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\household/census2022_household.csv
Saved household-level simulation for 2023 -> C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\household/census2023_household.csv
Saved household-level simulation for 2024 -> C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\household/census2024_household.csv
Saved household-level simulation for 2025 -> C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Upda

In [368]:
# Define the years you want to include
years = range(2021, 2031)

# List to hold second rows
second_rows = []

for year in years:
    file_path = os.path.join(folder_path, "household", f"census{year}_household.csv")
    
    try:
        # Read the full CSV and select the second row (index 1)
        second_row = pd.read_csv(file_path).iloc[[0]].copy()
        
        # Add a 'year' column
        second_row['year'] = year
        
        # Append to the list
        second_rows.append(second_row)
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue
    except IndexError:
        print(f"File {file_path} does not have a second row.")
        continue

# Combine all second rows into one DataFrame
summary_df = pd.concat(second_rows, ignore_index=True)

# Define the columns you want to keep
columns_to_keep = [
    'year', 'HH_ID','SHELCO', 'NOS', 'REPAIR', 'BEDRM', 'TENUR', 'PRESMORTG', 'AGEGRP' 'quintile', 
    'totalincome', 'totaltransfers', 'MRKINC', 'GTRFS', "WEIGHT"
]

# Keep only the desired columns (ignore missing ones)
summary_df = summary_df[[col for col in columns_to_keep if col in summary_df.columns]]

# Set 'year' as the index
summary_df.set_index('year', inplace=True)

# Transpose the DataFrame
summary_transposed = summary_df.transpose()

# Save the transposed DataFrame to Excel
output_path = os.path.join(folder_path, "household", "census_trace_hh.xlsx")
summary_transposed.to_excel(output_path)

print(f"Created transposed summary Excel file with years as columns at {output_path}")


Created transposed summary Excel file with years as columns at C:/Users/mgordon/OneDrive - Financial Accountability Office of Ontario/FA2404 Housing and Homelessness Update/Data\Microsimulations\household\census_trace_hh.xlsx


In [369]:


# Define the years and quintiles
years = range(2021, 2031)
quintiles = [1, 2, 3]

# Dictionaries to store weighted sums and weighted counts by year and quintile
weighted_HH_MRKINC = {year: {} for year in years}
weighted_HH_transfers = {year: {} for year in years}
weighted_HH_totalincome = {year: {} for year in years}  # NEW for totalincome
weighted_HH_counts = {year: {} for year in years}  # To store sum of weights per quintile

# Loop through each year and calculate weighted sums & counts by quintile
for year in years:
    file_path = os.path.join(folder_path, "household", f"census{year}_household.csv")
    
    try:
        df = pd.read_csv(file_path)
        
        # Ensure household-level aggregation by taking the first value per HH_ID
        df_household = df.groupby("HH_ID").agg(
            {
                "MRKINC": "first",
                "totaltransfers": "first",
                "totalincome": "first",  # NEW: Add totalincome
                "WEIGHT": "first",
                "quintile": "first"
            }
        ).reset_index()

        for q in quintiles:
            df_q = df_household[df_household["quintile"] == q]
            
            # Calculate weighted sums
            weighted_HH_MRKINC[year][q] = (df_q["MRKINC"] * df_q["WEIGHT"]).sum()
            weighted_HH_transfers[year][q] = (df_q["totaltransfers"] * df_q["WEIGHT"]).sum()
            weighted_HH_totalincome[year][q] = (df_q["totalincome"] * df_q["WEIGHT"]).sum()  # NEW: Total income
            
            # Calculate total weight (sum of weights) per quintile
            weighted_HH_counts[year][q] = df_q["WEIGHT"].sum()
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue

# Compute per-household weighted averages
avg_HH_MRKINC = {
    year: {q: (weighted_HH_MRKINC[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}
avg_HH_transfers = {
    year: {q: (weighted_HH_transfers[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}
avg_HH_totalincome = {  # NEW: Compute avg for total income
    year: {q: (weighted_HH_totalincome[year][q] / weighted_HH_counts[year][q]) if weighted_HH_counts[year][q] != 0 else None for q in quintiles}
    for year in years
}

# Print Year-over-Year Growth for Household Market Income (MRKINC)
print("\n--- Year-over-Year Per-Household MRKINC Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):  # Start from 2023 since we compare against the previous year
        prev = avg_HH_MRKINC.get(year - 1, {}).get(q)
        curr = avg_HH_MRKINC.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH MRKINC YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH MRKINC YoY Growth: N/A")

# Print Year-over-Year Growth for Household Total Transfers
print("\n--- Year-over-Year Per-Household Total Transfers Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):
        prev = avg_HH_transfers.get(year - 1, {}).get(q)
        curr = avg_HH_transfers.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH Transfers YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH Transfers YoY Growth: N/A")

# Print Year-over-Year Growth for Household Total Income
print("\n--- Year-over-Year Per-Household Total Income Growth by Quintile ---")
for q in quintiles:
    print(f"\nQuintile {q}:")
    for year in range(2022, 2031):
        prev = avg_HH_totalincome.get(year - 1, {}).get(q)
        curr = avg_HH_totalincome.get(year, {}).get(q)
        growth = ((curr - prev) / prev) * 100 if prev is not None and prev != 0 else None
        print(f"Year {year} | Avg HH Total Income YoY Growth: {growth:.2f}%" if growth is not None else f"Year {year} | Avg HH Total Income YoY Growth: N/A")



--- Year-over-Year Per-Household MRKINC Growth by Quintile ---

Quintile 1:
Year 2022 | Avg HH MRKINC YoY Growth: 1.24%
Year 2023 | Avg HH MRKINC YoY Growth: 5.77%
Year 2024 | Avg HH MRKINC YoY Growth: 0.73%
Year 2025 | Avg HH MRKINC YoY Growth: 2.57%
Year 2026 | Avg HH MRKINC YoY Growth: 2.21%
Year 2027 | Avg HH MRKINC YoY Growth: 2.93%
Year 2028 | Avg HH MRKINC YoY Growth: 2.69%
Year 2029 | Avg HH MRKINC YoY Growth: 2.62%
Year 2030 | Avg HH MRKINC YoY Growth: 2.58%

Quintile 2:
Year 2022 | Avg HH MRKINC YoY Growth: 3.94%
Year 2023 | Avg HH MRKINC YoY Growth: 2.87%
Year 2024 | Avg HH MRKINC YoY Growth: 6.32%
Year 2025 | Avg HH MRKINC YoY Growth: 2.71%
Year 2026 | Avg HH MRKINC YoY Growth: 2.39%
Year 2027 | Avg HH MRKINC YoY Growth: 3.11%
Year 2028 | Avg HH MRKINC YoY Growth: 2.83%
Year 2029 | Avg HH MRKINC YoY Growth: 2.73%
Year 2030 | Avg HH MRKINC YoY Growth: 2.67%

Quintile 3:
Year 2022 | Avg HH MRKINC YoY Growth: 6.81%
Year 2023 | Avg HH MRKINC YoY Growth: 2.32%
Year 2024 | Avg H

In [370]:
# Define the years and benefits of interest
years = [2024, 2025]
benefits = ["social", "cpp", "ei", "oas", "child", "totaltransfers"]

# Dictionary to store weighted sums and counts per year
weighted_sums = {year: {b: 0 for b in benefits} for year in years}
total_weights = {year: 0 for year in years}

# Process each year's data
for year in years:
    file_path = os.path.join(folder_path, f"census{year}.csv")
    
    try:
        df = pd.read_csv(file_path)
        
        # Group by household (HH_ID) and aggregate:
        df_household = df.groupby("HH_ID").agg(
            {b: "sum" for b in benefits} | {"WEIGHT": "first", "quintile": "first"}  # Sum benefits, keep first weight & quintile
        ).reset_index()
        
        # Filter for Quintile 1
        df_q1 = df_household[df_household["quintile"] == 1]
        
        # Compute weighted sums for each benefit
        for b in benefits:
            weighted_sums[year][b] = (df_q1[b] * df_q1["WEIGHT"]).sum()
        
        # Compute total weight for Quintile 1
        total_weights[year] = df_q1["WEIGHT"].sum()
    
    except FileNotFoundError:
        print(f"File not found for year {year}: {file_path}")
        continue

# Compute weighted averages
weighted_averages = {
    year: {b: (weighted_sums[year][b] / total_weights[year]) if total_weights[year] > 0 else None for b in benefits}
    for year in years
}

# Print results
print("\n--- Weighted Average Household Benefit Values for Quintile 1 ---")
for year in years:
    print(f"\nYear {year}:")
    for b in benefits:
        value = weighted_averages[year][b]
        print(f"  {b.capitalize()} Benefit: {value:,.2f}" if value is not None else f"  {b.capitalize()} Benefit: N/A")


--- Weighted Average Household Benefit Values for Quintile 1 ---

Year 2024:
  Social Benefit: 6,023.20
  Cpp Benefit: 4,590.85
  Ei Benefit: 459.09
  Oas Benefit: 6,041.56
  Child Benefit: 1,248.71
  Totaltransfers Benefit: 18,363.40

Year 2025:
  Social Benefit: 6,177.51
  Cpp Benefit: 4,763.92
  Ei Benefit: 484.70
  Oas Benefit: 6,238.02
  Child Benefit: 1,310.39
  Totaltransfers Benefit: 18,974.55


In [371]:
import pandas as pd

# Set input path and load data
input_base_path = folder_path
df = pd.read_csv(os.path.join(input_base_path, "census2021.csv"))

# Step 1: Flag if individual is a non-permanent resident (NPR)
df['is_npr'] = df['IMMSTAT'] == 3

# Step 2: Group by HH_ID to summarize household composition
hh_summary = df.groupby('HH_ID')['is_npr'].agg(
    total_members='count',
    n_nprs='sum'
)
hh_summary['n_non_nprs'] = hh_summary['total_members'] - hh_summary['n_nprs']

# Step 3: Classify household type
def classify_household(row):
    if row['n_nprs'] == row['total_members']:
        return 'All NPRs'
    elif row['n_nprs'] == 0:
        return 'No NPRs'
    else:
        return 'Mixed'

hh_summary['household_type'] = hh_summary.apply(classify_household, axis=1)

# Step 4: Merge back to assign each person their household type
df = df.merge(hh_summary[['household_type']], on='HH_ID', how='left')

# Step 5: Filter to only NPRs
npr_only = df[df['IMMSTAT'] == 3]

# Step 6: Weighted count of NPRs by household type
weighted_npr_distribution = npr_only.groupby('household_type')['WEIGHT'].sum().round(0).astype(int)

# Display result
print("📊 Weighted count of NPRs by household type:")
print(weighted_npr_distribution)


📊 Weighted count of NPRs by household type:
household_type
All NPRs    170107
Mixed        56133
Name: WEIGHT, dtype: int64


In [372]:


# Get NPR household IDs (All NPRs or Mixed)
npr_household_ids = hh_summary[hh_summary['household_type'].isin(['All NPRs', 'Mixed'])].index.tolist()

# Convert to DataFrame
npr_hh_df = pd.DataFrame(npr_household_ids, columns=['HH_ID'])

# Export to CSV
output_path = os.path.join(folder_path, "npr_household_ids.csv")
npr_hh_df.to_csv(output_path, index=False)

print(f"✅ Exported NPR household IDs to: {os.path.normpath(output_path)}")


✅ Exported NPR household IDs to: C:\Users\mgordon\OneDrive - Financial Accountability Office of Ontario\FA2404 Housing and Homelessness Update\Data\Microsimulations\npr_household_ids.csv
